In [1]:
import sys
sys.path.append("../src/") 
import os

import model_manipulation as mm
import cobra
import cplex 
import libsbml
import pandas as pd
import copy
from pathlib import Path
import memote
import csv
import pytest


In [2]:
#Read 2-cell model
model = cobra.io.read_sbml_model("../model/ios2164_2cell.xml")
#Estimate inf
inf = 1e6
model.solver = 'gurobi'


Set parameter Username
Academic license - for non-commercial use only - expires 2022-06-12


In [3]:

#Define linear relationship between PPFD and Cellular maintainance costs
#This formula comes from Topfer et al (2020) where she defined NGAM in a linear relationship with incident light
def compute_ngam_atp(ppfd):
    v_atp = 0.0049*ppfd + 2.7851
    return v_atp

In [4]:
#Initialize medium so as to induce Photoautotrophic conditions (i.e. no Sucrose import)
model_media = mm.read_medium_csv('../misc/photo_medium.csv', model)
    
model_media['EX_no3(e)'] = inf
model_media['EX_nh4(e)'] = 0
model_media['EX_h2o(e)'] = inf

model_media['EX_h(e)'] = 0
model.medium =model_media

model.medium


{'EX_co2(e)': 1000000,
 'EX_h2o(e)': 1000000.0,
 'EX_no3(e)': 1000000.0,
 'EX_o2(e)': 1000000,
 'EX_pi(e)': 1000000,
 'EX_so4(e)': 1000000,
 'EX_so3(e)': 1000000,
 'EX_h2s(e)': 1000000,
 'EX_fe2(e)': 1000000,
 'EX_fe3(e)': 1000000,
 'EX_mg2(e)': 1000000,
 'EX_photonVis(e)': 1000000}

In [5]:
# #Add constraints to model
#This code block contains constraints that would simulate the assimilation rates of bs and m cells in a two-cell system (such as those seen near the midvein region of rice leaves)
# #BS photon flux must be the same/less than M flux (Adapted from B&B, 2019)
# photon_import = model.reactions.get_by_id("EX_photonVis(e)")

BS_photon_import = model.reactions.PRISM_white_LED_BS
M_photon_import = model.reactions.PRISM_white_LED_M

#Set photon flux ratio between M/BS cell to be 1:1 or greater 

photon_flux = mm.set_fix_flux_ratio({M_photon_import.id:10, BS_photon_import.id:1},model)

# photon_flux = model.problem.Constraint(M_photon_import.flux_expression 
#                                        - BS_photon_import.flux_expression,
#                                        lb = 0, ub = inf)
model.add_cons_vars(photon_flux)

#Set photon flux such that 

co2tex_m = model.reactions.CO2tex_M
co2tex_bs = model.reactions.CO2tex_BS

#CO2 Uptake must be constrained to A (Net assimilation rate) which is around 29 +- 1.2 umol CO2 m-2 s-1 for wild type rice
#Lower bound based  on Dark CO2 uptake rates from Ermakova et al (2020)
#CO2 BS intake must be limited owing to its position in the leaf tissue

# # #CO2 intake needs to be mostly in the M cells. Not sure of the exact value:
# # Values from Von Caemmerer (2000) in intermediate C3-C4 photosynthetic plants is 1 mmol CO2 m-2 s-1 per 20 CO2 m-2 s-1, which puts it at 20:1 ratio
# # #try checking Danila et al. (2016) on ratio of surface area of M cell to BS cell
# # #I'll assume that the ratio is at 20:1 in the meantime, meaning that there is minimal gas exchange into the BS cell that would equate to around ~1 umol CO2 m-2s-1
co2_ratio_cons = mm.set_fix_flux_ratio({co2tex_m.id:10, co2tex_bs.id:1},model)
model.add_cons_vars(co2_ratio_cons)

# #Automatic na palang nalalagay siya as constraint

# #First is the BS and M Cell constraints
# #What if I restrict yung import ng O2?

# #Let's try restricting O2 to the value from Lakshmanan et al. (2016) = 3.312 mmol O2 m-2 s-1
# #Actually this value is also reflected in Von Caemmerer's "Modelling C3 metabolism" (pg. 85)
o2tex_bs = model.reactions.O2tex_BS
o2tex_m = model.reactions.O2tex_M

o2_ratio_cons = mm.set_fix_flux_ratio({o2tex_m.id:10, o2tex_bs.id:1},model)
model.add_cons_vars(o2_ratio_cons)

# bs_o2_cons = model.problem.Constraint(0.047*co2tex_bs.flux_expression 
#                                       - 1 * o2tex_bs.flux_expression, lb=0, ub=1000)
# model.add_cons_vars(bs_o2_cons)

In [6]:
# #This code block contains constraints specific for enzyme rate constraints
#This approach is derived from Bogart & Myers (2016) where they constrained the enzyme rate 
#fluxes in each of the 2-cell segments to a specific upper bound while keeping the lower bound
#At 0.


# #PEPC
wt_pepc = 2.4 #umol m-2 s-1
wt_mdh = 11.18 #umol m-2 s-1
wt_nadp_me = 0.14 #umol m-2 s-1
wt_ppdk = 0.31 #umol m-2 s-1
wt_CA = 7.5 #umol m-2  s-1 bar-1 (Constrained to CO2 amounting to 400-500 mbar) (1 bar = 15.74 umol m-2 s-1 CA activity)

#PEPC constraint (Reaction id: PPCc)
pepc_BS = model.reactions.PPCc_BS
pepc_M = model.reactions.PPCc_M

wt_pepc_cons = model.problem.Constraint(pepc_BS.flux_expression 
                                        + pepc_M.flux_expression, 
                                        lb = 0, ub = wt_pepc)
wt_pepc_cons.name = 'wt_pepc_cons'
model.add_cons_vars(wt_pepc_cons)

#PPDK constraints (Reaction id: PPDKS) (note that this is found in the chloroplast?) 
#Not detected via immunolocalization but enzyme activity is detected

ppdks_BS = model.reactions.PPDKs_BS
ppdks_M = model.reactions.PPDKs_M
wt_ppdks_cons = model.problem.Constraint(ppdks_BS.flux_expression 
                                         + ppdks_M.flux_expression, 
                                         lb = 0, ub = wt_ppdk)
wt_ppdks_cons.name = 'wt_ppdks_cons'
model.add_cons_vars(wt_ppdks_cons)

#Malate Dehydrogenase 
mdh2c_M = model.reactions.MDH2c_M
mdh2c_BS = model.reactions.MDH2c_BS
mdh2s_M = model.reactions.MDH2s_M
mdh2s_BS = model.reactions.MDH2s_BS

wt_mdh_cons = model.problem.Constraint(mdh2c_M.flux_expression 
                                       + mdh2c_BS.flux_expression 
                                       + mdh2s_M.flux_expression 
                                       + mdh2s_BS.flux_expression, 
                                       lb= 0, ub=wt_mdh)
wt_mdh_cons.name = "wt_mdh_cons"
model.add_cons_vars(wt_mdh_cons)


#NADP-ME (Since no signal is detected in WT, no locational constraints are imposed)
#Let's see if I can force it to have a small amount of flux 
nadp_me_M = model.reactions.MDHys_M
nadp_me_BS = model.reactions.MDHys_BS

wt_nadpme_cons = model.problem.Constraint(nadp_me_M.flux_expression
                                         + nadp_me_BS.flux_expression,
                                         lb= 0, ub=wt_nadp_me)
wt_nadpme_cons.name = "wt_nadpme"
model.add_cons_vars(wt_nadpme_cons)
#Weird, it causes the activity of RBCL to significantly vary from the actual values


#I should add constraints for Carbonic Anhydrase
#I should constrain it to 0.4 ubar, which would constitute ambient CO2 partial pressure
#
#HCO3Es
#HCO3Ec
#HCO3Em
#Flux is reversible so constraints are bi-directional


hco3es_m = model.reactions.HCO3Es_M.flux_expression
hco3ec_m = model.reactions.HCO3Ec_M.flux_expression
hco3em_m = model.reactions.HCO3Em_M.flux_expression
hco3es_bs = model.reactions.HCO3Es_BS.flux_expression
hco3ec_bs = model.reactions.HCO3Ec_BS.flux_expression
hco3em_bs = model.reactions.HCO3Em_BS.flux_expression

ca_cons = model.problem.Constraint(hco3es_m + hco3ec_m + hco3em_m 
                                   + hco3es_bs + hco3ec_bs + hco3em_bs,
                                  lb = -wt_CA, ub = 0)
ca_cons.name = 'Carbonic_anhydrase_constraint'
model.add_cons_vars(ca_cons)
#Rbcl constaints

#Let's try to add Rubisco flux ratio

#Retrieve flux expressions oof each RBCl reaction
rbpc_M = model.reactions.RBPCs_M.flux_expression
rbpc_BS = model.reactions.RBPCs_BS.flux_expression
rbpo_M = model.reactions.RBPOs_M.flux_expression
rbpo_BS = model.reactions.RBPOs_BS.flux_expression

#Constraint such that it is limited to 132 umol m-2 s-1
rbcl_vcmax_cons = model.problem.Constraint(rbpc_M + rbpc_BS, lb = 0, ub= 132)

model.add_cons_vars(rbcl_vcmax_cons)

#Constraints for rbcl flux such that v_c/v_o = 3 or more


rbcl_vcvo = model.problem.Constraint(3*(rbpo_M + rbpo_BS) 
                                     - 1*(rbpc_M + rbpc_BS),
                                     lb=0,ub=1000)
rbcl_vcvo.name = 'rbcl_vc/vo_ratio'
model.add_cons_vars(rbcl_vcvo)

#What if I simply constrained that of the M cell one to 3:1?
#This constraint is pretty good actually. 
#This allows the system to be set at a specific Vc/Vo rate while still allowing local variation 
#wherein Rubisco may act in an uncoupled fashion and may have favorable internal vc/vo rates.


In [7]:
# #This code block is to set a constraint such that M-to-BS cell NGAM ratio is 10-to-1 
# #Similar to what Moreno-Villena (2021) (preprint) had done 

ngam_atp_m = mm.get_rxn(model, 'ngam_atp_c_M')
ngam_atp_bs = mm.get_rxn(model, 'ngam_atp_c_BS')
ngam_atp_m.bounds = (0,1000)
ngam_atp_bs.bounds = (0,1000)
ngam_ratio = mm.set_fix_flux_ratio({ngam_atp_m.id:10, ngam_atp_bs.id:1}, model)
ngam_ratio.name = 'ngam_BS/M_ratio'
model.add_cons_vars(ngam_ratio)



This code block is to add the constraints for ngam nadphox, which for some reason doesn't save when it was generated using the curation script


In [8]:
#Retrieve NGAM reactions
ngam_nadphox_c_M = mm.get_rxn(model, 'ngam_nadphox_c_M')
ngam_nadphox_s_M = mm.get_rxn(model, 'ngam_nadphox_s_M')
ngam_nadphox_m_M = mm.get_rxn(model, 'ngam_nadphox_m_M')
ngam_nadphox_c_BS = mm.get_rxn(model, 'ngam_nadphox_c_BS')
ngam_nadphox_s_BS = mm.get_rxn(model, 'ngam_nadphox_s_BS')
ngam_nadphox_m_BS = mm.get_rxn(model, 'ngam_nadphox_m_BS')


#Set Fixed fluxes
nadphox_c_s_M = mm.set_fix_flux_ratio({ngam_nadphox_c_M.id:1, ngam_nadphox_s_M.id:1},model)
nadphox_c_s_M.name = "nadphox_cs_ratio_M"
nadphox_s_m_M = mm.set_fix_flux_ratio({ngam_nadphox_s_M.id:1, ngam_nadphox_m_M.id:1}, model)
nadphox_s_m_M.name = "nadphox_sm_ratio_M"

nadphox_c_s_BS = mm.set_fix_flux_ratio({ngam_nadphox_c_BS.id:1, ngam_nadphox_s_BS.id:1},model)
nadphox_c_s_BS.name = "nadphox_cs_ratio_BS"
nadphox_s_m_BS = mm.set_fix_flux_ratio({ngam_nadphox_s_BS.id:1, ngam_nadphox_m_BS.id:1}, model)
nadphox_s_m_BS.name = "nadphox_sm_ratio_BS"

#Add constraints
model.add_cons_vars(nadphox_c_s_M)
model.add_cons_vars(nadphox_s_m_M)
model.add_cons_vars(nadphox_c_s_BS)
model.add_cons_vars(nadphox_s_m_BS)


In [9]:

#Retrieve flux expressionns
fex_nadphox_c_M =  mm.get_flux_exp(model, ngam_nadphox_c_M)
fex_nadphox_s_M = mm.get_flux_exp(model, ngam_nadphox_s_M)
fex_nadphox_m_M = mm.get_flux_exp(model, ngam_nadphox_m_M)

fex_nadphox_c_BS =  mm.get_flux_exp(model, ngam_nadphox_c_BS)
fex_nadphox_s_BS =  mm.get_flux_exp(model, ngam_nadphox_s_BS)
fex_nadphox_m_BS =  mm.get_flux_exp(model, ngam_nadphox_m_BS)

fex_atp_c_M = mm.get_flux_exp(model, ngam_atp_m)
fex_atp_c_BS =  mm.get_flux_exp(model, ngam_atp_bs)

#Set the constraint between ATP:NADPH NGAM to 3:1
nadphox_atpase = model.problem.Constraint(3*(fex_nadphox_c_M + fex_nadphox_s_M + fex_nadphox_m_M
                                                   + fex_nadphox_c_BS + fex_nadphox_s_BS + fex_nadphox_m_BS) 
                                     - 1*(fex_atp_c_M + fex_atp_c_BS),
                                     lb=0,ub=0)
nadphox_atpase.name = "nadphox_atpase_ratio"
model.add_cons_vars(nadphox_atpase)


In [85]:
# This code block is to test whether a single iteration works. Maybe I should implement the kinetic constraints by Von Caemmerer to obtain proper 
#proper values? I think that would be a better approach, I think.


with model:
                                                                                                                                                                                                                                                                                                                      
    model_media['EX_photonVis(e)'] = 100
    model_media['EX_co2(e)'] = 29
    model_media['EX_o2(e)'] = 2.26#2.2618
    model.medium = model_media
    
#     #Set NGAM
    ngam_value = compute_ngam_atp(0)
    ngam_cons = model.problem.Constraint(fex_atp_c_M + 
                                        fex_atp_c_BS, lb=ngam_value, ub=ngam_value)
    model.add_cons_vars(ngam_cons)
#Constrain O2 intake to 3.312 mmol O2 gcw -1 d-1 ~~ 2.2618 umol O2 m-2 s-1 (from Lakshmanan et al. 2016)
    #Change objective function to mature leaf
    model.reactions.get_by_id('Straw_Biomass_M').objective_coefficient = 0
    model.reactions.get_by_id('Straw_Biomass_BS').objective_coefficient = 0
    mm.get_rxn(model,'DM_Phloem_BS').objective_coefficient = 1
    sample_fluxes = cobra.flux_analysis.pfba(model).fluxes
    #Remove thermodynamically infeasible loops
    sample_fluxes = cobra.flux_analysis.loopless_solution(model, sample_fluxes)
    
    sample_fluxes_df = sample_fluxes.to_frame()


sample_fluxes
    


,fluxes,reduced_costs
Coleoptile_Biomass_M,0.0,19014.113220
Straw_Biomass_M,0.0,19482.429776
ACCOAC_OSAc_M,0.0,-1.000000
ACCOAC_OSAr_M,0.0,-1.000000
ACPC_OSs_M,0.0,-1.000000
...,...,...
g3p_pd,0.0,2.000000
3pg_pd,0.0,3.000000
glyclt_pd,0.0,2.000000
acon-C_pd,0.0,-1.000000


In [86]:
#This code  block gives a snapshot of the relevant fluxes on each of the cell types based on the saved sample_fluxes values above
print('rbcl M cell: ', sample_fluxes['RBPCs_M'], 'rbcl BS cell: ',sample_fluxes['RBPCs_BS'])
print('rbcl M cell (photorespiration)', sample_fluxes['RBPOs_M'], 'rbcl BS cell (PR)', sample_fluxes['RBPOs_BS'])
print('vc/vo M:', sample_fluxes['RBPCs_M']/sample_fluxes['RBPOs_M'], 'vc/vo BS:', sample_fluxes['RBPCs_BS']/sample_fluxes['RBPOs_BS'])
print('PEPC M', sample_fluxes['PPCc_M'], 'PEPC BS', sample_fluxes['PPCc_BS'])
print('Carbonic Anhydrase (Cytosolic) M', sample_fluxes['HCO3Ec_M'], 'Carbonic Anhydrase (Cytosolic) BS', sample_fluxes['HCO3Ec_BS'])
print('NADP-ME M', sample_fluxes['MDHys_M'], 'NADP-ME BS', sample_fluxes['MDHys_BS'])
print('Biomass M: ', sample_fluxes['Straw_Biomass_M'], 'Biomass BS', sample_fluxes['Straw_Biomass_BS'])
print('Phloem M: ', sample_fluxes['DM_Phloem_M'], 'Phloem BS', sample_fluxes['DM_Phloem_BS'])
print('co2 consumption M', sample_fluxes[co2tex_m.id], 'co2 consumption BS', sample_fluxes[co2tex_bs.id])
print('o2 consumption M', sample_fluxes['O2tex_M'], 'o2 consumption BS', sample_fluxes['O2tex_BS'])
print('Photosystem II M', sample_fluxes['PSIINC_M'], 'PSII BS', sample_fluxes['PSIINC_BS'])
print('PSI M', sample_fluxes['PSIMR_M'], 'PSI BS', sample_fluxes['PSIMR_BS'])
print('PPFD M: ', sample_fluxes['PRISM_white_LED_M'], 'PPFD BS: ', sample_fluxes['PRISM_white_LED_BS'])

pd_rxn = [x for x in model.reactions if "pd" in x.id and "h2o" not in x.id]
pd_abs_flux = 0
for pds in pd_rxn:
    pd_abs_flux += abs(sample_fluxes[pds.id])
    
print('pd_abs_flux: ', pd_abs_flux)

rbcl M cell:  5.644835144765079 rbcl BS cell:  -1.4510628306110544e-11
rbcl M cell (photorespiration) 1.8816117149168559 rbcl BS cell (PR) 0.0
vc/vo M: 3.000000000007712 vc/vo BS: -inf
PEPC M 0.12404253065848342 PEPC BS 0.0013740207953194528
Carbonic Anhydrase (Cytosolic) M -0.009009377345935054 Carbonic Anhydrase (Cytosolic) BS 0.0
NADP-ME M 0.0 NADP-ME BS 0.0
Biomass M:  0.0 Biomass BS 0.0
Phloem M:  0.0 Phloem BS 0.09404360486362269
co2 consumption M 0.5005642119523402 co2 consumption BS 0.05005642119523402
o2 consumption M 2.0545454545454542 o2 consumption BS 0.20545454545454545
Photosystem II M 8.717872557685752 PSII BS 0.6815960123403327
PSI M 12.710555080613164 PSI BS 1.4131816688163985
PPFD M:  90.9090909090909 PPFD BS:  9.090909090909092
pd_abs_flux:  0.901973916714362


<ipython-input-86-e573d06bebb5>:4: RuntimeWarning: divide by zero encountered in double_scalars
  print('vc/vo M:', sample_fluxes['RBPCs_M']/sample_fluxes['RBPOs_M'], 'vc/vo BS:', sample_fluxes['RBPCs_BS']/sample_fluxes['RBPOs_BS'])


In [45]:
#This is to show a single instance of the script shown above

from tabulate import tabulate
print(tabulate(sample_fluxes_df, headers='keys', tablefmt='psql'))

+-----------------------+--------------+-------------------+
|                       |       fluxes |     reduced_costs |
|-----------------------+--------------+-------------------|
| Coleoptile_Biomass_M  |  0           |   99487.6         |
| Straw_Biomass_M       |  0           |   96887.6         |
| ACCOAC_OSAc_M         |  0           |      -1           |
| ACCOAC_OSAr_M         |  0           |      -1           |
| ACPC_OSs_M            |  0           |      -1           |
| ACPC_OSm_M            |  0           |      -1           |
| FAC_Osc_M             |  0           |      -1           |
| FAC_Osr_M             |  0           |      -1           |
| EX_co2(e)             |  0           |    -606.078       |
| EX_h2o(e)             |  5.64677     |     -29.9477      |
| EX_h(e)               |  0           |     124.052       |
| EX_no3(e)             | -0.00342027  |  -87780.4         |
| EX_o2(e)              | -2.26        |    1014.47        |
| EX_pi(e)              

In [87]:
# #This code block is to generate flux values thru the network given a set of constraints to PPFD

# ppfd_solns_2cell_fba = pd.DataFrame(index=list(i.id for i in model.reactions))
# ppfd_solns_2cell_fva  = pd.DataFrame(index=list(i.id for i in model.reactions))

# #Iterate from range 0 to 2000 w/ increments of 50 (max ppfd)
   
# for i in range(0,2100,50):
#     with model:
#         ppfd = i 
#         #Set medium to change photon flux and other things and re-add to model
#         model_media['EX_photonVis(e)'] = ppfd
#         model_media['EX_co2(e)'] = 29
#         model_media['EX_o2(e)'] = 2.2618 #umol/m^2 s O2
#         model.medium = model_media

#         model.reactions.get_by_id('Straw_Biomass_M').objective_coefficient = 1
#         model.reactions.get_by_id('Straw_Biomass_BS').objective_coefficient = 1



#         #perform pFBA & convert to dataframe
#         tc_soln = cobra.flux_analysis.pfba(model).to_frame()
# #         tc_summary = model.summary(fva=0.95).to_frame()
#         tc_soln = pd.DataFrame({ppfd:tc_soln['fluxes']}, index=list(tc_soln.index))
#         print("PPFD:", ppfd, " RBPC_M cell flux:", tc_soln[ppfd]['RBPCs_M'])
#         #Append to Dataframe
#         ppfd_solns_2cell_fba[ppfd] = tc_soln
#        # ppfd_solns_2cell_fva[ppfd] = 

# #Code now works

In [75]:
ppfd_solns_2cell_fba = pd.DataFrame(index=list(i.id for i in model.reactions))
ppfd_solns_2cell_fva  = pd.DataFrame(index=list(i.id for i in model.reactions))

#Iterate from range 0 to 2000 w/ increments of 50 (max ppfd)

for i in range(100,2100,50):
    
    #Generate instance of model every iteration
    with model:
        ppfd = i 
        #Set ATP NGAM value to linear PPFD formula

         #Set medium to change photon flux and other things and re-add to model
        model_media['EX_photonVis(e)'] = ppfd
        model_media['EX_co2(e)'] = 29
        model_media['EX_o2(e)'] = 2.2618 #umol/m^2 s O2 (converted from 3.312 mmol/dcw hr)
        model.medium = model_media

        #Set NGAM value
        ngam_value = compute_ngam_atp(i)
        ngam_cons = model.problem.Constraint(fex_atp_c_M + 
                                            fex_atp_c_BS, lb=ngam_value, ub=ngam_value)
        model.add_cons_vars(ngam_cons)


        #Change objective function to mature leaf
        model.reactions.get_by_id('Straw_Biomass_M').objective_coefficient = 0
        model.reactions.get_by_id('Straw_Biomass_BS').objective_coefficient = 0
        mm.get_rxn(model,'DM_Phloem_BS').objective_coefficient = 1

        #Optimize then check optimality
        solution = model.optimize()

        if solution.status == 'optimal':
            sample_fluxes = cobra.flux_analysis.pfba(model).fluxes
            #Remove thermodynamically infeasible loops via CycleFreeFlux algorithm
            sample_fluxes = cobra.flux_analysis.loopless_solution(model, sample_fluxes)
            #Generate dataframe
            sample_fluxes = sample_fluxes.to_frame()

            sample_fluxes = pd.DataFrame({ppfd:sample_fluxes['fluxes']}, index=list(sample_fluxes.index))
            print("PPFD:", ppfd, " RBPC_M cell flux:", sample_fluxes[ppfd]['RBPCs_M'])
            #Append to Dataframe
            ppfd_solns_2cell_fba[ppfd] = sample_fluxes
            
out = open('../flux_results/ppfd_2cell_solns_with_CycleFreeFlux.tsv', 'w')
ppfd_solns_2cell_fba.to_csv(out, sep='\t')
out.close()
print("Successfully saved PPFD-out (W/ CCF!)")

PPFD: 100  RBPC_M cell flux: 6.099135819364534
PPFD: 150  RBPC_M cell flux: 9.30597247825852
PPFD: 200  RBPC_M cell flux: 12.248729164235343
PPFD: 250  RBPC_M cell flux: 15.295667438842004
PPFD: 300  RBPC_M cell flux: 18.40984423366545
PPFD: 350  RBPC_M cell flux: 21.397371328121668
PPFD: 400  RBPC_M cell flux: 24.384456253560106
PPFD: 450  RBPC_M cell flux: 27.36777294057578
PPFD: 500  RBPC_M cell flux: 30.35108963581827
PPFD: 550  RBPC_M cell flux: 33.334406325999154
PPFD: 600  RBPC_M cell flux: 36.317723033792646
PPFD: 650  RBPC_M cell flux: 39.301039732119285
PPFD: 700  RBPC_M cell flux: 42.27638030885729
PPFD: 750  RBPC_M cell flux: 45.18930490267714
PPFD: 800  RBPC_M cell flux: 48.07237920160538
PPFD: 850  RBPC_M cell flux: 51.03916325436875
PPFD: 900  RBPC_M cell flux: 54.02797741671177
PPFD: 950  RBPC_M cell flux: 57.0132460874825
PPFD: 1000  RBPC_M cell flux: 60.00124063486669
PPFD: 1050  RBPC_M cell flux: 62.94646661934598
PPFD: 1100  RBPC_M cell flux: 65.88166241448006
PPFD:

In [78]:
ppfd_solns_2cell_fba_noccf = pd.DataFrame(index=list(i.id for i in model.reactions))
#Iterate from range 0 to 2000 w/ increments of 50 (max ppfd)

for i in range(0,2100,50):
    
    #Generate instance of model every iteration
    with model:

        #Set ATP NGAM value to linear PPFD formula

        #Set medium to change photon flux and other things and re-add to model
        ppfd = i 
        model_media['EX_photonVis(e)'] = ppfd
        model_media['EX_co2(e)'] = 29
        model_media['EX_o2(e)'] = 2.2618 #umol/m^2 s O2 (converted from 3.312 mmol/dcw hr)
        model.medium = model_media

        #Set NGAM value
        ngam_value = compute_ngam_atp(i)
        ngam_cons = model.problem.Constraint(fex_atp_c_M + 
                                            fex_atp_c_BS, lb=ngam_value, ub=ngam_value)
        model.add_cons_vars(ngam_cons)
        
        #Change objective function to mature leaf
        model.reactions.get_by_id('Straw_Biomass_M').objective_coefficient = 0
        model.reactions.get_by_id('Straw_Biomass_BS').objective_coefficient = 0
        mm.get_rxn(model,'DM_Phloem_BS').objective_coefficient = 1

        #Optimize then check optimality
        solution = model.optimize()

        if solution.status == 'optimal':
            sample_fluxes = cobra.flux_analysis.pfba(model).fluxes
            sample_fluxes = sample_fluxes.to_frame()
            
            #Generate dataframe
            sample_fluxes = pd.DataFrame({ppfd:sample_fluxes['fluxes']}, index=list(sample_fluxes.index))
            print("PPFD:", ppfd, " RBPC_M cell flux:", sample_fluxes[ppfd]['RBPCs_M'])
            #Append to larger Dataframe
            ppfd_solns_2cell_fba_noccf[ppfd] = sample_fluxes

            
            
out = open('../flux_results/ppfd_2cell_solns_no_CycleFreeFlux.tsv', 'w')
ppfd_solns_2cell_fba_noccf.to_csv(out, sep='\t')
out.close()
print('Successfully saved PPFD-out (No CCF!)')

PPFD: 100  RBPC_M cell flux: 6.0991358217177325
PPFD: 150  RBPC_M cell flux: 9.305972478910068
PPFD: 200  RBPC_M cell flux: 12.248729164607626
PPFD: 250  RBPC_M cell flux: 15.295667429850475
PPFD: 300  RBPC_M cell flux: 18.409844231701676
PPFD: 350  RBPC_M cell flux: 21.39737132813358
PPFD: 400  RBPC_M cell flux: 24.384456242354005
PPFD: 450  RBPC_M cell flux: 27.367772940217712
PPFD: 500  RBPC_M cell flux: 30.351089638082303
PPFD: 550  RBPC_M cell flux: 33.33440633594702
PPFD: 600  RBPC_M cell flux: 36.31772303381611
PPFD: 650  RBPC_M cell flux: 39.30103973168923
PPFD: 700  RBPC_M cell flux: 42.27638031823119
PPFD: 750  RBPC_M cell flux: 45.18930490293923
PPFD: 800  RBPC_M cell flux: 48.07237920139962
PPFD: 850  RBPC_M cell flux: 51.03916330558443
PPFD: 900  RBPC_M cell flux: 54.0279774159196
PPFD: 950  RBPC_M cell flux: 57.01324608885525
PPFD: 1000  RBPC_M cell flux: 60.001240634980185
PPFD: 1050  RBPC_M cell flux: 62.94646661928801
PPFD: 1100  RBPC_M cell flux: 65.88166241441226
PPF

In [21]:
#Constraining O2 consumption to around 3.312 forces water to be consumed by the system, unlike when O2 is unconstrained.
#However, the same behavior when it comes to CO2 assimilation still is being produced.
#Constraining O2 consumption also forces Nitrate consumption to be constrained to realistic levels (when compared to non-constrained O2 where consumption was 10-fold )
#I suspect that this is definitely due to how PPFD flux is structured. 
#A cop-out to this would be to double PPFD per run and measuring PPFD flux in the specific reactions instead.

In [22]:
#I can probably do the benchmarking right now. The model behaves quite interestingly actually

In [23]:
#Note: Turning off other prism reactions (red, green, blue, etc) while retaining only white light reactions lowers net carbon assimilation
#I think this is in consideration on how white light is partitioned to several wavelengths, which prevents it from being fully utilized.

In [24]:
#Other artifacts as of the moment include the following:
#model doesn't produce any Oxygen. Maybe I should add a specific demand reaction for it?
#Based on FVA, the model now produces some oxygen via the demand reaction. 

In [25]:
#Model also facilitates transfer of metabolites unlike before.

In [26]:
#Questions:
#Should I restrict Nutrient flow to the Bundle Sheath Cell only or should I allow flux to both the M and BS cell?
#Apparently kasi dito only the Bundle sheath cells produce any biomass
#This is to ensure that only CO2 and Light flux are the only limiting constraints to the system.